In [ ]:
# Import necessary libraries
import os
import cv2
import json
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from PIL import Image, ImageDraw

In [ ]:
train_set_1_path = r"C:\Users\Charitha\Downloads\training set 1.zip"
train_set_2_path = r"C:\Users\Charitha\Downloads\training set 2.zip"
test_set_path = r"C:\Users\Charitha\Downloads\test.zip"

In [ ]:
# Function to load images and annotations from Training Set 1
def load_training_set_1(path):
    images = []
    annotations = []
    for filename in os.listdir(path):
        if filename.endswith('.jpg'):
            img_path = os.path.join(path, filename)
            images.append(cv2.imread(img_path))
            annotation_path = os.path.join(path, filename.replace('.jpg', '.json'))
            with open(annotation_path, 'r') as f:
                annotations.append(json.load(f))
    return images, annotations

In [ ]:
# Function to visualize a sample image with bounding box from Training Set 1
def visualize_training_set_1(image, annotation):
    ymin, xmin, ymax, xmax = annotation['bbox']
    image = cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (255, 0, 0), 2)
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title("Training Set 1 - Bounding Box")
    plt.show()


In [ ]:
# Function to visualize a sample image with text annotation from Training Set 2
def visualize_training_set_2(image, annotation):
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(f"Training Set 2 - License Plate: {annotation}")
    plt.show()

In [ ]:
# Analyze annotations in Training Set 1
def analyze_annotations_1(annotations):
    bbox_data = []
    for ann in annotations:
        bbox_data.append(ann['bbox'])
    df = pd.DataFrame(bbox_data, columns=['ymin', 'xmin', 'ymax', 'xmax'])
    sns.pairplot(df)
    plt.title("Bounding Box Coordinates Distribution")
    plt.show()

In [ ]:
# Analyze annotations in Training Set 2
def analyze_annotations_2(annotations):
    lengths = [len(ann) for ann in annotations]
    plt.hist(lengths, bins=20)
    plt.title("Length of License Plate Text Annotations")
    plt.xlabel("Number of Characters")
    plt.ylabel("Frequency")
    plt.show()

In [ ]:
!pip install pandas opencv-python matplotlib torch torchvision tensorflow scikit-learn


In [ ]:
# Load annotations
annotations = pd.read_csv('annotations.csv')

In [ ]:
# Load images
images = {}
for filename in os.listdir('images'):
    img = cv2.imread(os.path.join('images', filename))
    if img is not None:
        images[filename] = img

In [ ]:

print("Data loaded successfully!")

In [ ]:
# Display a sample image with its bounding box
sample_image = images['sample.jpg']
annotation = annotations[annotations['filename'] == 'sample.jpg']

ymin, xmin, ymax, xmax = annotation[['ymin', 'xmin', 'ymax', 'xmax']].values[0]

plt.imshow(cv2.cvtColor(sample_image, cv2.COLOR_BGR2RGB))
plt.gca().add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin, linewidth=2, edgecolor='r', facecolor='none'))
plt.show()


In [ ]:
# Parse bounding box coordinates
annotations['bbox'] = annotations.apply(lambda row: (row['xmin'], row['ymin'], row['xmax'], row['ymax']), axis=1)
annotations.head()


In [ ]:
# Basic statistics for bounding boxes
bbox_stats = annotations[['xmin', 'ymin', 'xmax', 'ymax']].describe()
print(bbox_stats)

# Frequency of characters in license plates
char_freq = pd.Series(''.join(annotations['text'].values)).value_counts()
print(char_freq)


In [ ]:
# Plot bounding box size distribution
bbox_sizes = annotations.apply(lambda row: (row['xmax'] - row['xmin']) * (row['ymax'] - row['ymin']), axis=1)
plt.hist(bbox_sizes, bins=50)
plt.xlabel('Bounding Box Size')
plt.ylabel('Frequency')
plt.show()

# Plot character frequency distribution
plt.bar(char_freq.index, char_freq.values)
plt.xlabel('Character')
plt.ylabel('Frequency')
plt.show()


In [ ]:
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# Load a pre-trained Faster R-CNN model and modify it for our use case
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
num_classes = 2  # Background + license plate
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# Define training loop (simplified)
from torch.utils.data import DataLoader, Dataset
import numpy as np

class LicensePlateDataset(Dataset):
    def __init__(self, annotations, images):
        self.annotations = annotations
        self.images = images

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        img_name = self.annotations.iloc[idx]['filename']
        image = self.images[img_name]
        bbox = self.annotations.iloc[idx]['bbox']
        target = {
            'boxes': torch.tensor([bbox], dtype=torch.float32),
            'labels': torch.tensor([1], dtype=torch.int64)
        }
        return image, target

train_dataset = LicensePlateDataset(annotations, images)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for images, targets in train_loader:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
    print(f"Epoch {epoch} Loss: {losses.item()}")

print("Training complete!")


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Reshape, Dense, LSTM

# Example of a simple CRNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 128, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Reshape((32, 64*64)),
    LSTM(128, return_sequences=True),
    LSTM(128),
    Dense(36, activation='softmax')  # 36 classes for alphanumeric characters
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Dummy data for example purposes
train_images = np.random.rand(100, 32, 128, 1)
train_labels = np.random.randint(36, size=(100, 32))
val_images = np.random.rand(20, 32, 128, 1)
val_labels = np.random.randint(36, size=(20, 32))

# Train the model
model.fit(train_images, train_labels, epochs=10, validation_data=(val_images, val_labels))
print("Training complete!")


In [ ]:
def recognize_license_plate(image):
    # Detect license plate
    model.eval()
    with torch.no_grad():
        boxes, scores, labels = model([image.to(device)])
    box = boxes[0][0].cpu().numpy().astype(int)
    xmin, ymin, xmax, ymax = box

    # Crop detected license plate
    plate_image = image[ymin:ymax, xmin:xmax]
    plate_image_resized = cv2.resize(plate_image, (128, 32))

    # Recognize text from cropped license plate
    text = model_ocr.predict(np.expand_dims(plate_image_resized, axis=0))
    return text

# Example usage
test_image = images['test.jpg']
recognized_text = recognize_license_plate(test_image)
print(recognized_text)


In [ ]:
def iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

# Calculate IoU for a test image
detected_box = [xmin, ymin, xmax, ymax]  # From the detection model
true_box = annotations[annotations['filename'] == 'test.jpg'][['xmin', 'ymin', 'xmax', 'ymax']].values[0]

print("IoU:", iou(detected_box, true_box))


In [ ]:
def character_accuracy(predicted_text, true_text):
    correct_chars = sum(p == t for p, t in zip(predicted_text, true_text))
    return correct_chars / len(true_text)

def word_accuracy(predicted_text, true_text):
    return int(predicted_text == true_text)

# Calculate accuracy for a test image
recognized_text = "ABC123"  # From the recognition model
true_text = "ABC123"  # From the ground truth

print("Character Accuracy:", character_accuracy(recognized_text, true_text))
print("Word Accuracy:", word_accuracy(recognized_text, true_text))


In [ ]:
total_accuracy = []
for filename in test_images:
    test_image = images[filename]
    true_text = annotations[annotations['filename'] == filename]['text'].values[0]
    recognized_text = recognize_license_plate(test_image)
    
    total_accuracy.append(character_accuracy(recognized_text, true_text))

print("Overall Character Accuracy:", sum(total_accuracy) / len(total_accuracy))
